In [1]:
# As first_reg, but with seeking the difference between the label and the
# prediction given by bilinear interpolation.

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
class ZeroRegressor(_PredictorBase):
    """Predicts zero for labels."""
    
    use_tf = False
    
    @classmethod
    def _predict(cls, Xi, y):
        return np.zeros(y.shape)


class BilinearInterp(BilinearInterpBase):
    @classmethod
    def _predict(cls, Xi, y):
        returnval = []
        # Translation doesn't matter at this point so WLOG the fine grid is
        # around 0, 0. (cls._interp makes the same assumption; these assumptions
        # must be consistent)
        for point in fine_grid((0, 0)):
            returnval.append(cls._interp(Xi, point))
        return returnval
    

def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the feature is the solution on a coarse grid and the label is the 
    difference between solution on a fine grid and the prediction given by
    bilinear interpolation.
    """
    
    # Random solution to the CH equation
    c = np.random.uniform(0, 10)
    peakon = Peakon(c=c)
    # Random location
    t = np.random.uniform(0, 40)
    x = c * t + np.random.uniform(-5, 5)
    
    # Grids at the location
    cg = coarse_grid((t, x))
    fg = fine_grid((t, x))
    # Features: the solution on the coarse grid
    X = peakon.on_grid(cg)
    # Labels: the difference between the solution on the fine grid and the
    # prediction given by bilinear interpolation
    y = peakon.on_grid(fg)
    # Predict expects to be given a batch of elements, so we have to wrap
    # the input X, y in a list, and then extract the prediction from a list
    # of one element.
    predictor = BilinearInterp.predict(input_fn=lambda: ({'X': [X]}, [y]))
    y_prediction = next(predictor)[0]
    
    return X, y - y_prediction

In [5]:
# DNN hyperparameters
hidden_units = [2000] * 40
logits = 121  # = (_fine_grid_fineness.t + 1) * (_fine_grid_fineness.x + 1)
              # i.e. the number of fine grid points.
drop_rate = 0.4
gradient_clip = 5.0

# Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir = './saved_models/fifth_reg/'
layer_counter = [(k, sum(1 for _ in g)) for k, g in it.groupby(hidden_units)]
for layer_size, layer_repeat in layer_counter:
    model_dir += '{}x{}_'.format(layer_size, layer_repeat)
model_dir += '{}_'.format(logits)
model_dir += 'D0{}'.format(int(drop_rate * 10))
# model_dir += 'BS{}_'.format(batch_size)
# model_dir += 'BR{}'.format(batch_reuse)

k_init = tfi.truncated_normal(mean=0, stddev=0.05)
model = Sequential()
for units in hidden_units:
    model.add(tfla.Dense(units=units,
                         activation=tf.nn.elu,
                         kernel_initializer=k_init))
    model.add_train(tfla.Dropout(rate=drop_rate))
model.add(tfla.Dense(units=logits))
model.set_kwargs(model_dir=model_dir,
                 config=tfe.RunConfig(log_step_count_steps=log_steps))

In [6]:
dnn = model.compile(gradient_clip=gradient_clip)

INFO:tensorflow:Using config: {'_model_dir': './saved_models/fifth_reg/2000x40_121_D04', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f49c2700ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
if TRAIN:
    # Training hyperparameters
    batch_size = 50
    batch_reuse = 1  # See BatchData for an explanation of batch reuse.
    steps = 25000
    
    train_input_fn = BatchData(gen_one_data, batch_size, batch_reuse)
    dnn.train(input_fn=train_input_fn, max_steps=steps)
    
    batch_size = 250
    batch_reuse = 1  # See BatchData for an explanation of batch reuse.
    steps = 30000
    
    train_input_fn = BatchData(gen_one_data, batch_size, batch_reuse)
    dnn.train(input_fn=train_input_fn, max_steps=steps)
    
    batch_size = 500
    batch_reuse = 1  # See BatchData for an explanation of batch reuse.
    steps = 35000
    
    k_init = tfi.truncated_normal(mean=0, stddev=0.05)
    model = Sequential()
    for units in hidden_units:
        model.add(tfla.Dense(units=units,
                             activation=tf.nn.elu,
                             kernel_initializer=k_init))
        model.add_train(tfla.Dropout(rate=drop_rate))
    model.add(tfla.Dense(units=logits))
    model.set_kwargs(model_dir=model_dir,
                     config=tfe.RunConfig(log_step_count_steps=log_steps))
    
    dnn = model.compile(gradient_clip=gradient_clip)
    train_input_fn = BatchData(gen_one_data, batch_size, batch_reuse)
    dnn.train(input_fn=train_input_fn, max_steps=steps)

INFO:tensorflow:Skipping training since max_steps has already saved.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt-28779
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 28780 into ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt.
INFO:tensorflow:loss = 0.31777567, step = 28780
INFO:tensorflow:Saving checkpoints for 28846 into ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt.
INFO:tensorflow:Saving checkpoints for 28912 into ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt.
INFO:tensorflow:Saving checkpoints for 28977 into ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt.
INFO:tensorflow:Saving checkpoints for 29042 into ./saved_models/fifth_reg/2000x40_121_D04/model.ckpt.
INFO:tensorflow:Saving checkpoi

KeyboardInterrupt: 

In [ ]:
testing_data = gen_test_data(gen_one_data)

In [ ]:
test_regressor(testing_data, dnn)

In [ ]:
test_regressor(testing_data, ZeroRegressor)